In [ ]:
import os
os.chdir('/home/mohsen/projects/scarches/')

In [ ]:
import scanpy as sc
import scarches as sca
import seaborn as sns
import scIB as scib
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
def compute_metrics(latent_adata, adata, data_name, method, rqr=None, batch_key='study', label_key='cell_type'):
    latent_adata.obsm['X_pca'] = latent_adata.X
    print(adata.shape, latent_adata.shape)
    n_batches = len(adata.obs[batch_key].unique().tolist())
    
    scores = scib.metrics.metrics(adata, latent_adata, batch_key, label_key, 
                                  nmi_=True, ari_=True, silhouette_=True, pcr_=True, graph_conn_=True, 
                                  isolated_labels_=True, hvg_score_=False)
    scores = scores.T
    scores = scores[['NMI_cluster/label', 'ARI_cluster/label', 'ASW_label', 'ASW_label/batch', 
                     'PCR_batch', 'isolated_label_F1', 'isolated_label_silhouette', 'graph_conn']]
    
    ebm = sca.metrics.entropy_batch_mixing(latent_adata, batch_key, n_neighbors=15)
    knn = sca.metrics.knn_purity(latent_adata, label_key, n_neighbors=15)
    
    scores['EBM'] = ebm
    scores['KNN'] = knn
    scores['method'] = method
    scores['data'] = data_name
    scores['rqr'] = rqr / n_batches if rqr is not None else None
    
    return scores

In [ ]:
def compute_rqr_metrics(dataset):
    scores = None

    if dataset == 'mouse_brain':
        adata = sc.read(f'/home/mohsen/data/{dataset}/mouse_brain_subsampled_normalized_hvg.h5ad')
        batch_key, label_key = 'study', 'cell_type'
        ratios = [1, 2, 3]
    elif dataset == 'pancreas':
        adata = sc.read(f'/home/mohsen/data/{dataset}/{dataset}_normalized.h5ad')
        batch_key, label_key = 'study', 'cell_type'
        ratios = [1, 2, 3, 4]
    else:
        adata = sc.read(f'/home/mohsen/data/PBMC/Immune_ALL_human_wo_villani_rqr_normalized_hvg.h5ad')
        batch_key, label_key = 'condition', 'final_annotation'
        ratios = [1, 4, 8]
        
    scores = None
    for method in ['scvi', 'scanvi', 'CVAE', 'trVAE', 'DCA']:
        for ratio in ratios:
            print(f'**** Running for {method} with ratio = {ratio} ****')
            if method in ['CVAE', 'trVAE', 'DCA']:
                latent_adata = sc.read(f'/home/mohsen/data/scArches/ref_query_ratio/{method}/{dataset}/{ratio}/all.h5ad')
            elif method in ['scvi', 'scanvi']:
                if dataset == 'pbmc':
                    test_num = 1 + ratio // 4
                    figure_num = 6
                    latent_adata = sc.read(f'/home/mohsen/data/scArches/scVI_scANVI/figure_{figure_num}/{method}/test_{test_num}_first_cond/full_data.h5ad')
                else:
                    test_num = ratio
                    figure_num = 3
                    latent_adata = sc.read(f'/home/mohsen/data/scArches/scVI_scANVI/figure_{figure_num}/{method}/{dataset}/test_{test_num}_first_cond/full_data.h5ad')                    
                
                latent_adata.obs[batch_key] = latent_adata.obs['batch'].values
                latent_adata.obs[label_key] = latent_adata.obs['celltype'].values
                            
            df = compute_metrics(latent_adata, adata, dataset, method, ratio, batch_key, label_key)
            scores = pd.concat([scores, df], axis=0) if scores is not None else df
            
    if dataset in ['pancreas', 'mouse_brain']:
        for method in ['Harmony', 'Liger', 'Scanorama', 'Seurat', 'mnnCorrect', 'Conos']:
            print(f'**** Running for {method} ****')
            latent_adata = sc.read(f'/home/mohsen/data/scArches/Full Integration Results/{dataset}/{method}/result_adata.h5ad')
            df = compute_metrics(latent_adata, adata, dataset, method, None, batch_key, label_key)
            scores = pd.concat([scores, df], axis=0) if scores is not None else df
            
    return scores

In [ ]:
scores = compute_rqr_metrics('pbmc')
scores

In [ ]:
scores = compute_rqr_metrics('mouse_brain')